In [1]:
import os
import nda_yaml
from ccf.redcap import RedcapTable
from ccf.easy_yaml import EasyYaml

Y = EasyYaml()
tags = []

# Download NDA structures

In [2]:
if os.path.exists('nda') and not os.path.exists('.bk.nda'):
    os.rename('nda', '.bk.nda')

structures = 'scidv_pscyh01 diagpsx01 asr01 batbil01 bisbas01 bsc01 cbcl01 cbcl1_501 cbq01 deldisk01 drugscr01 eatq01 edinburgh_hand01 er4001 fenvs01 gales01 gbi01 gonogo01 ipaq01 lbadl01 leap01 mab01 mchq01 mctq01 medh01 mendt01 mmse01 moca01 neo_ffi_form_s_adult_200301 nffi01 pds01 phenx_su01 psqi01 ravlt01 saiq01 scan_debrief01 screentime01 sdq01 sleepdis01 socdem01 srs02 upps01 vision_tests01 vitals01 wais_iv_part101 wisc_v01 wppsiiv01 ysr01'.split(' ')
for struct in structures:
    nda_yaml.download(struct)

if os.path.exists('.bk.nda') and os.path.exists('nda'):
    !!meld .bk.nda/ nda/

In [2]:
%%time
for filename in os.listdir("nda/"):
    path = f"nda/{filename}"
    contents = Y(path)
    tags.extend([f"{x}\t{path}\t/^{x}:$/;\"\tnda" for x in contents.keys()])

CPU times: user 97.6 ms, sys: 6.72 ms, total: 104 ms
Wall time: 104 ms


Original time: **user** 14.5 s **sys:** 729 ms  **total:** 15.2 s

# Download Redcap definitions

In [4]:
if os.path.exists('nda') and not os.path.exists('.bk.definitions'):
    os.rename('definitions', '.bk.definitions')

structures = ['parent', 'teen', 'child', 'hcpa', 'ssaga', 'ksads', 'qint']
for struct in structures:
    RedcapTable.get_table_by_name(struct).download_datadictionary('definitions')

if os.path.exists('.bk.definitions') and os.path.exists('definitions'):
    !!meld .bk.definitions/ definitions/

In [3]:
%%time

for filename in os.listdir("definitions/"):
    path = f"definitions/{filename}"
    contents = Y(path)
    tags.extend([f"{x}\t{path}\t/^{x}:$/;\"\tredcap" for x in contents.keys()])
    

CPU times: user 173 ms, sys: 0 ns, total: 173 ms
Wall time: 172 ms


Original time: **user** 20.3 s **sys:** 126 ms  **total:** 20.4 s

# Write Tags
(To make it easy to jump to definitions in `vim`.)

In [5]:
with open('tags', 'w') as f:
    tags.sort()
    f.write('\n'.join(tags))